# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [470]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

import glob
import natsort
import pandas as pd
import numpy as np
from dateutil.parser import parse

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Data field names
Column names and the names of the variables we want to extract slightly vary between countries. Hence, first we define the column names of the _date_ column, _variable_ and _national total_ columns as well as the names of the fields for each country.

In [471]:
# Dataframe importing 
# Colums: country, month, averagy, new cases, death

# column names and interesting variables for each country
col_date = {"guinea" : "Date",
            "liberia": "Date",
            "sl" : "date"} # column name of the date column

col_name={"guinea" : "Description",
          "liberia": "Variable",
          "sl" : "variable"} # column name of the indicator variables

col_total = {"guinea":"Totals",
             "liberia":"National",
             "sl": "National"} # column name where the national totals per day are stored

col_counties={"guinea":["Conakry","Gueckedou","Macenta","Dabola","Kissidougou","Dinguiraye",
                        "Telimele","Boffa","Kouroussa","Siguiri","Pita","Nzerekore","Yomou",
                        "Dubreka","Forecariah","Kerouane","Coyah","Dalaba","Beyla","Kindia"],
              "liberia":["Bomi County","Bong County","Grand Kru","Lofa County","Margibi County",
                         "Maryland County","Montserrado County","Nimba County","River Gee County",
                         "RiverCess County","Sinoe County"],
              "sl":["Kailahun","Kenema","Kono","Kambia","Koinadugu","Bombali","Tonkolili","Port Loko",
                    "Pujehun","Bo","Moyamba","Bonthe","Western area urban","Western area rural"]}

var_death_new =  {"guinea" : ["New deaths registered today (confirmed)","New deaths registered"],
             "liberia": ["Newly reported deaths"],# "Total death/s in confirmed cases"
             "sl":["etc_new_deaths"]}

var_death_total =  {"guinea" : ["Total deaths of confirmed"],
             "liberia": ["Total death/s in confirmed cases"],# "Total death/s in confirmed cases"
             "sl":["death_confirmed"]}

var_cases_new = {"guinea" : ["New cases of confirmed"],
             "liberia": ["New case/s (confirmed)"],
             "sl":["new_confirmed"]}

var_cases_tot = {"guinea" : ["Total cases of confirmed"],
             "liberia": ["Total confirmed cases"],
             "sl":["cum_confirmed"]}


### Aggregate data frames
Now, we loop over all files and import them, applying necessary clean-up operations such as type conversions, comma stripping etc. For each file (day), we extract one row with the corresponding values and merge them in `df`.

In [472]:
dfs = [] # dataframes for each day
for c in ['guinea','liberia','sl']: #
    for file in sorted(glob.glob(DATA_FOLDER+"/ebola/"+c+"_data/*.csv")):
        df = pd.read_csv(file,index_col=None)
        df = df.fillna(0) # Fill NA values
        
        # Get Data
        columns=[col_date[c],col_name[c],col_total[c]]
        
        # Dates
        date = pd.to_datetime(df[1:2][col_date[c]]) # dates
        
        # New cases
        df_cases_new = df[df[col_name[c]].isin(var_cases_new[c])] # extract only relevant rows
        df_cases_new = df_cases_new[columns] # extract only relevant columns (date - name - total)
        df_cases_new[0:1][col_total[c]].values[0] = str(df_cases_new[0:1][col_total[c]].values[0]).replace(',','') # strip commas for 1000s
        df_cases_new[col_total[c]] = df_cases_new[col_total[c]].astype(int)     # convert to int (e.g., 10.0 or 10,0)
        cases_new = int(df_cases_new[0:1][col_total[c]]) # cases
        
        # Total cases
        df_cases_total = df[df[col_name[c]].isin(var_cases_tot[c])] # extract only relevant rows
        df_cases_total = df_cases_total[columns] # extract only relevant columns (date - name - total)
        df_cases_total[0:1][col_total[c]].values[0] = str(df_cases_total[0:1][col_total[c]].values[0]).replace(',','') # strip commas for 1000s
        df_cases_total[col_total[c]] = df_cases_total[col_total[c]].astype(int)     # convert to int (e.g., 10.0 or 10,0)
        cases_total = int(df_cases_total[0:1][col_total[c]]) # cases

        # New deaths
        df_deaths_new = df[df[col_name[c]].isin(var_death_new[c])] # extract only relevant rows
        df_deaths_new = df_deaths_new[columns] # extract only relevant columns (date - name - total)
        deaths_new = int(df_deaths_new[0:1][col_total[c]]) # deaths 
        
        # Total Deaths
        df_deaths_total = df[df[col_name[c]].isin(var_death_total[c])] # extract only relevant rows
        df_deaths_total = df_deaths_total[columns] # extract only relevant columns (date - name - total)
        deaths_total = int(df_deaths_total[0:1][col_total[c]]) # deaths 
        
        # Total Deaths by County
        df_deaths_total_county_sum = df[df[col_name[c]].isin(var_death_total[c])] # extract only relevant rows
        cols=list(set(df.columns.values).difference(columns))
        df_deaths_total_county_sum = df_deaths_total_county_sum[cols] # extract only relevant columns (date - name - total)
        for col in df_deaths_total_county_sum.columns:
            df_deaths_total_county_sum[col]=df_deaths_total_county_sum[col].astype(int)
        deaths_total_county_sum = np.nansum(df_deaths_total_county_sum.values)
        
        # new dataframe from data (one row per date)
        df = pd.DataFrame({"Country":c,"Date":date,"New Cases":cases_new,"Total Cases":cases_total,"New Deaths":deaths_new,"Total Deaths":deaths_total,"Total Deaths Counties":deaths_total_county_sum},index=None)
        dfs.append(df)
        
for i in range(len(dfs)-1): # concatenate all dataframes (rows)
    dfs[0] = pd.concat([dfs[0],dfs[i+1]])
df=dfs[0]

df=df.sort_values(by=['Country','Date']) # sort by country and date
df=df.set_index(['Country','Date']) # new index
df.head()

New Cases  New Deaths  Total Cases  Total Deaths  \
Country Date                                                           
guinea  2014-08-04          4           2          351           228   
        2014-08-26         10           5          490           292   
        2014-08-27         10           2          499           294   
        2014-08-30          9           5          533           324   
        2014-08-31         29           3          563           337   

                    Total Deaths Counties  
Country Date                               
guinea  2014-08-04                    228  
        2014-08-26                    292  
        2014-08-27                    267  
        2014-08-30                    324  
        2014-08-31                    337

There seems to be a discrepancy between the new deaths (confirmed) and the total deaths (confirmed), as the new cases /deaths don't add up to the cumsum of the total cases / deaths of the next date. Maybe the data doesn't come from the same source. Also, the total deaths don't always quite add up with the sum of deaths per county, which may be due to incomplete data.

In [473]:
df.tail()

New Cases  New Deaths  Total Cases  Total Deaths  \
Country Date                                                           
sl      2014-12-01         72           0         5978          1549   
        2014-12-04         37           3         6238          1648   
        2014-12-05         54           8         6292          1669   
        2014-12-06          0           0         6317          1708   
        2014-12-13          0           0         6638             0   

                    Total Deaths Counties  
Country Date                               
sl      2014-12-01                   1549  
        2014-12-04                   1648  
        2014-12-05                   1669  
        2014-12-06                   1708  
        2014-12-13                      0

### Data checking

Let's check the data whether the index is unique and whether there are still NaN values

In [474]:
# Data checking
print("Index unique:")
print(df.index.is_unique) # True
for col in df.columns:
    print(col+":")
    print(df[df[col].isnull()].size) # get NaN colums (all 0)
    
df=df.reset_index() # for next part

Index unique:
True
New Cases:
0
New Deaths:
0
Total Cases:
0
Total Deaths:
0
Total Deaths Counties:
0


### Aggregate by month

Now, we can regroup the data by month and sums each column by the month. Then, we rename the date column as month column 

In [460]:
# Aggregate by month

df_month = pd.DataFrame(df[["Country","Date","New Cases","New Deaths"]].groupby([df.Country,df.Date.dt.month]).mean()).reset_index()
df_month.columns=["Country","Month","Mean New Cases","Mean New Deaths"]
df_month = df_month.set_index(['Country','Month'])
df_month.round(2)

Mean New Cases  Mean New Deaths
Country Month                                 
guinea  8               12.40             3.40
        9               13.00             3.56
        10               6.00            15.00
liberia 6                2.14             2.00
        7                1.82             4.27
        8                5.44            23.22
        9                6.17            36.04
        10               1.36            28.04
        11               2.60            13.47
        12            1928.33             0.00
sl      8               18.55             0.00
        9               34.41             0.28
        10              56.71             3.54
        11              59.90             0.57
        12              32.60             2.20

It seems like Ebola peaked around September in Guinea, around August-October in Liberia and around October-November in Sierra Leone.

_suspicious_: no deaths in November and December in Liberia but many new cases. Maybe erroneous data or breakout of a new epidemia.

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

### DataFrame Merging
First we merge all data frames:

In [484]:
# Write your answer here

# Read in Excel files, concatenate them and add file name (MID1...9) for later merging
dfs=[]
for i in range(1,10):
    MID_tmp=pd.read_excel(DATA_FOLDER+'/microbiome/MID'+str(i)+'.xls', sheetname='Sheet 1', header=None,index_col=0)
    MID_tmp.columns=['Count']
    MID_tmp['BARCODE'] = "MID"+str(i)  # add column with file name
    # new dataframe from data
    dfs.append(MID_tmp)
        
for i in range(len(dfs)-1): # concatenate all dataframes
    dfs[0] = pd.concat([dfs[0],dfs[i+1]])

MID=dfs[0]
MID.index.name="Species"
MID.head()

,Count,BARCODE
Species,,
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7,MID1
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",2,MID1
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae Stygiolobus",3,MID1
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae Thermofilum",3,MID1
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae Methanocella",7,MID1


### Metadata import and Merging
We can now import the metadata and merge the files.

In [485]:
# read metadata
metadata = pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', sheetname='Sheet1',index_col=0)

# merge metadata and files
MID_merged = pd.merge(metadata, MID, left_index=True, right_on='BARCODE')
# print(MID_merged.index.is_unique) # True
MID_merged=MID_merged.fillna("unknown")
MID_merged=MID_merged.reset_index().set_index(['BARCODE','Species']) # new index
MID_merged.head()

GROUP  \
BARCODE Species                                                                  
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  EXTRACTION CONTROL   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  EXTRACTION CONTROL   

                                                             SAMPLE  Count  
BARCODE Species                                                             
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      7  
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      2  
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  unknown      3  
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  unknown      3  
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  unknown      7

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here